# Food 101

This is a Jupyter Notebook containing context and calculations for our Machine Learning end-of-year project.

*Main goal:* We want to train a machine learning model that - given an image of a plate of food - is able to tell us what kind of dish it is.

*Strategies:*
*   kNN algorithm (with different parameters)
*   `DecisionTreeClassifier` from sklearn
*   XGBoost?

## Dataset

This [Food 101](https://www.kaggle.com/datasets/dansbecker/food-101) or [Food images - Food 101](https://www.kaggle.com/datasets/kmader/food41?select=images)  dataset from Kaggle contains nearly 10000 images of dishes divided into 101 types of food.

In [ ]:
import tf as tf
!pip install -q gdown httpimport
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle

Add kaggle.json file from your kaggle profile to ~/.kaggle/


In [ ]:
# Downloading data
!kaggle datasets download 'dansbecker/food-101' -p /path/data

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
# Unzipping data from Kaggle
!unzip /path/data/food-101.zip -d data/

unzip:  cannot find or open /path/data/food-101.zip, /path/data/food-101.zip.zip or /path/data/food-101.zip.ZIP.


In [ ]:
# Imports

import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf

## kNN algorithm


### Complete Python script for kNN

**NOTE:** Since the model requires a lot of computing power it should probably not be run in Google Colab. Adjusting the following parameters will have an effect on execution time:

In [ ]:
how_many_classes = 10  # out of 101
picture_size = 48      # the image will be resized to a square of this size by OpenCV

In [ ]:
if __name__ == '__main__':
    # Data loading
    pictures = '/food-101/images'

    # Selects x first categories/food classes
    classes = [x[:-1:] for x in open('food-101/meta/classes.txt', 'r').readlines()][:how_many_classes:]
    labels = [x[:-1:] for x in open('food-101/meta/labels.txt', 'r').readlines()][:how_many_classes:]
    # Conversion from string to id
    conversion = dict()
    for index in range(how_many_classes):
        conversion[classes[index]] = index

    test_path = [x[:-1:] for x in open('food-101/meta/test.txt', 'r').readlines()]
    train_path = [x[:-1:] for x in open('food-101/meta/train.txt', 'r').readlines()]

    test_x = []
    train_x = []
    test_y = []
    train_y = []

    # Creating test_x and test_y based on chosen classes
    for path in test_path:
        path_class = path.split('/')[0]
        if path_class in classes:
            img = cv.imread(f'food-101/images/{path}.jpg')
            if img.shape != (picture_size, picture_size, 3):
                img = cv.resize(img, (picture_size, picture_size))
            test_x.append(img)
            test_y.append(conversion[path_class])
        else:
            break

    # Creating train_x and train_y based on chosen classes
    for path in train_path:
        path_class = path.split('/')[0]
        if path_class in classes:
            img = cv.imread(f'food-101/images/{path}.jpg')
            if img.shape != (picture_size, picture_size, 3):
                img = cv.resize(img, (picture_size, picture_size))
            train_x.append(img)
            train_y.append(conversion[path_class])
        else:
            break

    # cv.imshow('Picture', test_x[0])
    # cv.imshow('Resized', cv.resize(test_x[0], (50, 50)))
    # cv.waitKey(3000)

    tf.compat.v1.disable_eager_execution()

    # Prepare data for knn algorithm
    knn_train_y = np.eye(how_many_classes)[train_y]
    knn_test_y = np.eye(how_many_classes)[test_y]
    train_x = np.array([picture.flatten() for picture in train_x])
    test_x = np.array([picture.flatten() for picture in test_x])

    train_x = (train_x - train_x.min(0)) / train_x.ptp(0)
    test_x = (test_x - test_x.min(0)) / test_x.ptp(0)

    print('Separated into subsets')

    # Knn alg
    features = len(test_x[0])
    k = 5
    x_new_train = tf.compat.v1.placeholder(shape=[None, features], dtype=tf.float32)
    y_new_train = tf.compat.v1.placeholder(shape=[None, len(knn_test_y[0])], dtype=tf.float32)
    x_new_test = tf.compat.v1.placeholder(shape=[None, features], dtype=tf.float32)

    manht_distance = tf.reduce_sum(tf.abs(tf.subtract(x_new_train, tf.expand_dims(x_new_test, 1))), axis=2)

    print('Distance done!')

    _, top_k_indices = tf.nn.top_k(tf.negative(manht_distance), k=k)
    top_k_labels = tf.gather(y_new_train, top_k_indices)

    predictions_sumup = tf.reduce_sum(top_k_labels, axis=1)
    make_prediction = tf.argmax(predictions_sumup, axis=1)

    sess = tf.compat.v1.Session()
    # outcome_prediction = sess.run(make_prediction, feed_dict={x_new_train: train_x, x_new_test: test_x,
    #                                                           y_new_train: knn_train_y})

    accuracy = 0
    batch_size = 2

    for i in range(len(knn_test_y) // batch_size):
        res = sess.run(make_prediction,
                       feed_dict={x_new_train: train_x, x_new_test: test_x[i * batch_size:(i + 1) * batch_size],
                                  y_new_train: knn_train_y})
        for pred, actual in zip(res, knn_test_y[i * batch_size:(i + 1) * batch_size]):
            if pred == np.argmax(actual):
                accuracy += 1

    print("Accuracy rate:", accuracy / len(knn_test_y))

## Random Forest

We want to create a classifier similar to [this one](https://github.com/pooji0401/Image-Classification-System-using-Decision-Trees/blob/master/Image%20Classification%20System.ipynb) - ...

In [ ]:
# Additional imports
from sklearn import tree
from sklearn.metrics import accuracy_score

### Complete Python script for decision trees

**NOTE:** Since the model requires a lot of computing power it should probably not be run in Google Colab (and depending on the program parameters it may still take a few hours).

In [ ]:
how_many_classes = 10  # out of 101
picture_size = 48      # the image will be resized to a square of this size by OpenCV

In [ ]:
# Data loading
pictures = '/food-101/images'

# Selects x first categories/food classes
classes = [x[:-1:] for x in open('food-101/meta/classes.txt', 'r').readlines()][:how_many_classes:]
labels = [x[:-1:] for x in open('food-101/meta/labels.txt', 'r').readlines()][:how_many_classes:]

# Conversion from string to id
conversion = dict()
for index in range(how_many_classes):
    conversion[classes[index]] = index

test_path = [x[:-1:] for x in open('food-101/meta/test.txt', 'r').readlines()]
train_path = [x[:-1:] for x in open('food-101/meta/train.txt', 'r').readlines()]

test_x = []
train_x = []
test_y = []
train_y = []

# Creating test_x and test_y based on chosen classes
for path in test_path:
    path_class = path.split('/')[0]
    if path_class in classes:
        img = cv.imread(f'food-101/images/{path}.jpg')
        if img.shape != (picture_size, picture_size, 3):
            img = cv.resize(img, (picture_size, picture_size))
        test_x.append(img)
        test_y.append(conversion[path_class])
    else:
        break

# Creating train_x and train_y based on chosen classes
for path in train_path:
    path_class = path.split('/')[0]
    if path_class in classes:
        img = cv.imread(f'food-101/images/{path}.jpg')
        if img.shape != (picture_size, picture_size, 3):
            img = cv.resize(img, (picture_size, picture_size))
        train_x.append(img)
        train_y.append(conversion[path_class])
    else:
        break

D_train = [_.flatten() for _ in train_x]
D_test = [_.flatten() for _ in test_x]

clf = tree.DecisionTreeClassifier()
clf = clf.fit(D_train, train_y)

D_train = np.array(D_train)

n_nodes = clf.tree_.node_count
print(n_nodes)
print(clf.tree_.max_depth)

pred = clf.predict(D_test)

print('Accuracy rate:', accuracy_score(pred, test_y))


## Notes

*Ideas*:
*   keep the current way of calculating the distance, but only take every 10th or so pixel into account (since things close to one another tend to be similar)
*   introduce noise to the data (greying some random pixels)
*   consider only the inner 90% of the frame (in order to analyze the food itself rather than the place setting)

